In [25]:
import torch
from diffusers import StableDiffusionPipeline

In [26]:
pipe = StableDiffusionPipeline.from_pretrained(
    "j-min/reco_sd14_coco",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

Loading pipeline components...:  57%|█████▋    | 4/7 [00:00<00:00,  4.59it/s]An error occurred while trying to fetch /home/mliu/.cache/huggingface/hub/models--j-min--reco_sd14_coco/snapshots/11a062da5a0a84501047cb19e113f520eb610415/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /home/mliu/.cache/huggingface/hub/models--j-min--reco_sd14_coco/snapshots/11a062da5a0a84501047cb19e113f520eb610415/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /home/mliu/.cache/huggingface/hub/models--j-min--reco_sd14_coco/snapshots/11a062da5a0a84501047cb19e113f520eb610415/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /home/mliu/.cache/huggingface/hub/models--j-min--reco_sd14_coco/snapshots/11a062da5a0a84501047cb19e113f520eb610415/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 1

In [27]:
def create_reco_prompt(
        caption: str = '',
        phrases=[],
        boxes=[],
        normalize_boxes=True,
        image_resolution=512,
        num_bins=1000,
):
    """
    method to create ReCo prompt

    caption: global caption
    phrases: list of regional captions
    boxes: list of regional coordinates (unnormalized xyxy)
    """

    SOS_token = '<|startoftext|>'
    EOS_token = '<|endoftext|>'

    box_captions_with_coords = []

    box_captions_with_coords += [caption]
    box_captions_with_coords += [EOS_token]

    for phrase, box in zip(phrases, boxes):

        if normalize_boxes:
            box = [float(x) / image_resolution for x in box]

        # quantize into bins
        quant_x0 = int(round((box[0] * (num_bins - 1))))
        quant_y0 = int(round((box[1] * (num_bins - 1))))
        quant_x1 = int(round((box[2] * (num_bins - 1))))
        quant_y1 = int(round((box[3] * (num_bins - 1))))

        # ReCo format
        # Add SOS/EOS before/after regional captions
        box_captions_with_coords += [
            f"<bin{str(quant_x0).zfill(3)}>",
            f"<bin{str(quant_y0).zfill(3)}>",
            f"<bin{str(quant_x1).zfill(3)}>",
            f"<bin{str(quant_y1).zfill(3)}>",
            SOS_token,
            phrase,
            EOS_token
        ]

    text = " ".join(box_captions_with_coords)
    return text

In [28]:
prompt1 = "A box contains six donuts with varying types of glazes and toppings. <|endoftext|> <bin514> <bin575> <bin741> <bin765> <|startoftext|> chocolate donut. <|endoftext|> <bin237> <bin517> <bin520> <bin784> <|startoftext|> dark vanilla donut. <|endoftext|> <bin763> <bin575> <bin988> <bin745> <|startoftext|> donut with sprinkles. <|endoftext|> <bin234> <bin281> <bin524> <bin527> <|startoftext|> donut with powdered sugar. <|endoftext|> <bin515> <bin259> <bin767> <bin514> <|startoftext|> pink donut. <|endoftext|> <bin753> <bin289> <bin958> <bin506> <|startoftext|> brown donut. <|endoftext|>"

In [29]:
caption2 = "a photo of bus and boat; boat is left to bus."
phrases2 = ["a photo of a bus.", "a photo of a boat."]
boxes2 = [[0.702, 0.404, 0.927, 0.601], [0.154, 0.383, 0.311, 0.487]]
prompt2 = create_reco_prompt(caption2, phrases2, boxes2, normalize_boxes=False)

In [30]:
caption3 = "Six ramens with varying types of toppings."
phrases3 = ["pork ramen.", "chicken ramen.", "vegetable ramen.", "beef ramen.",
           "vegetable ramen.", "chicken ramen."]
boxes3 = [[263.68, 294.912, 380.544, 392.832], [121.344, 265.216, 267.392, 401.92], [391.168, 294.912, 506.368, 381.952],
         [120.064, 143.872, 268.8, 270.336], [264.192, 132.928, 393.216, 263.68], [386.048, 148.48, 490.688, 259.584]]
prompt3 = create_reco_prompt(caption3, phrases3, boxes3)

In [31]:
generated_image = pipe(
    prompt3,
    num_images_per_prompt=1,
    guidance_scale=4
).images[0]

100%|██████████| 50/50 [00:15<00:00,  3.30it/s]


In [32]:
generated_image.save("samples/g1.png", format="PNG")